In [1]:
import pandas as pd

def _url_to_img_tag(path):
    return f"""<img
    style="width: 100px; height: 100px; object-fit: contain;"
            src="{path}"/>"""


def styled(df: pd.DataFrame) -> pd.DataFrame:
    return df.style.format({'main_image': _url_to_img_tag,
                            'image_url': _url_to_img_tag,
                            'image_url_lhs': _url_to_img_tag,
                            'image_url_rhs': _url_to_img_tag})

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

search_template_src = """
{{!
  This is an Elasticsearch query expressed as a Mustache template. Its uploaded as a 'search template' and
  Elasticsearch uses it as kind of stored query that can be executed with different parameters. It allows
  us to 'program' Elasticsearch with query logic we can test independent of the query-side application (ie liaison)

  See https://www.elastic.co/guide/en/elasticsearch/reference/current/search-template.html
}}
{
  "query": {
    "bool": {
      {{! lexical scoring components - showing candidates with All matches }}
      "should": [
        {"bool": {
          "filter": [
            {"bool": {
              "filter": [
                {"range": {"min_price": {"gte": {{min_price}}}}}
              ]}},
            {"bool": {
              "filter": [
                {"range": {"max_price": {"lte": {{max_price}}}}}
              ]}},
            {"bool": {
              "should": [
                {{#categories}}
                {"match_phrase": {"category": "{{value}}"}} {{^last}},{{/last}}
                {{/categories}}
               ]}},
            {"bool": {
              "filter": [
                {{#departments}}
                {"match": {"department": "{{value}}"}} {{^last}},{{/last}}
                {{/departments}}
              ]}},
            {"bool": {
              "filter": [
                {{#brands}}
                {"match": {"brand_name.en": "{{value}}"}} {{^last}},{{/last}}
                {{/brands}}
              ]}},
            {"bool": {
              "should": [
                {{#sizes}}
                {"match": {"standard_sizes": "{{value}}"}} {{^last}},{{/last}}
                {{/sizes}}
              ]}}

            {{#has_option_ids}}
            ,{"ids":
               {"values": [
               {{#option_ids}}
               "{{value}}" {{^last}},{{/last}}
               {{/option_ids}}
             ]}}
            {{/has_option_ids}}
          ],
          "must": [
            {"multi_match": {
              "query": "{{keywords}}",
              "fields": ["name.en", "description.en", "brand_name.en"],
              "type": "cross_fields",
              "minimum_should_match": "100%"
            }}
          ]
        }}
      {{! image vector component - high precision matches, boosted higher }}
        {{#image_vector}}
        ,{"knn": {
          "field": "image_embedding",
          "boost": 1000,
          "k": 10,
          "filter": {
            {{! annoying - repeated lexical filters for prefiltering before vector search}}
            "bool": {
              "filter": [
                {"multi_match": {
                  "query": "{{keywords}}",
                  "fields": ["name.en", "description.en", "brand_name.en"],
                  "type": "cross_fields",
                  "minimum_should_match": "100%"
                }},
                {"bool": {
                  "filter": [
                    {"range": {"min_price": {"gte": {{min_price}}}}}
                  ]}},
                {"bool": {
                  "filter": [
                    {"range": {"max_price": {"lte": {{max_price}}}}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#categories}}
                    {"match_phrase": {"category": "{{value}}"}} {{^last}},{{/last}}
                    {{/categories}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#departments}}
                    {"match": {"department": "{{value}}"}} {{^last}},{{/last}}
                    {{/departments}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#brands}}
                    {"match": {"brand_name.en": "{{value}}"}} {{^last}},{{/last}}
                    {{/brands}}
                  ]}},
                  {"bool": {
                    "should": [
                      {{#sizes}}
                      {"match": {"standard_sizes": "{{value}}"}} {{^last}},{{/last}}
                      {{/sizes}}
                    ]}}
                 {{#has_option_ids}}
                 ,{"ids":
                    {"values": [
                    {{#option_ids}}
                    "{{value}}" {{^last}},{{/last}}
                    {{/option_ids}}
                  ]}}
                 {{/has_option_ids}}
              ]
            }
          },
          "query_vector": {{image_vector}}
        }}
        {{/image_vector}}
      {{! image vector component - lower precision matches }}
        {{#image_vector}}
        ,{"knn": {
          "field": "image_embedding",
          "boost": 100,
          "k": 10,
          "filter": {
            {{! repeated lexical filters for prefiltering before vector search}}
            "bool": {
              "filter": [
                {"multi_match": {
                  "query": "{{keywords}}",
                  "fields": ["name.en", "description.en", "brand_name.en"],
                  "type": "cross_fields",
                  "minimum_should_match": "1"
                }},
                {"bool": {
                  "filter": [
                    {"range": {"min_price": {"gte": {{min_price}}}}}
                  ]}},
                {"bool": {
                  "filter": [
                    {"range": {"max_price": {"lte": {{max_price}}}}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#categories}}
                    {"match_phrase": {"category": "{{value}}"}} {{^last}},{{/last}}
                    {{/categories}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#departments}}
                    {"match": {"department": "{{value}}"}} {{^last}},{{/last}}
                    {{/departments}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#brands}}
                    {"match": {"brand_name.en": "{{value}}"}} {{^last}},{{/last}}
                    {{/brands}}
                  ]}},
                  {"bool": {
                    "should": [
                      {{#sizes}}
                      {"match": {"standard_sizes": "{{value}}"}} {{^last}},{{/last}}
                      {{/sizes}}
                    ]}}
                 {{#has_option_ids}}
                 ,{"ids":
                    {"values": [
                    {{#option_ids}}
                    "{{value}}" {{^last}},{{/last}}
                    {{/option_ids}}
                  ]}}
                 {{/has_option_ids}}
              ]
            }
          },
          "query_vector": {{image_vector}}
        }}
        {{/image_vector}}
     ]
}}}
"""

script_spec = {"script": {"lang": "mustache", "source": search_template_src}}
es.put_script(body=script_spec, id="my-search-template")

ObjectApiResponse({'acknowledged': True})

In [3]:
import json

template_params = {
    "params": {
        "keywords": "search keywords",
        "image_vector": "[1,2,3]",
        "min_price": 0,
        "max_price": 100000000,
        "categories": [{"value": "shirt", "last": True}]
    }
}
templ_out = es.render_search_template(id="my-search-template", body=template_params)['template_output']

print(json.dumps(templ_out, indent=2))

{
  "query": {
    "bool": {
      "should": [
        {
          "bool": {
            "filter": [
              {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "min_price": {
                          "gte": 0
                        }
                      }
                    }
                  ]
                }
              },
              {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "max_price": {
                          "lte": 100000000
                        }
                      }
                    }
                  ]
                }
              },
              {
                "bool": {
                  "should": [
                    {
                      "match_phrase": {
                        "category": "shirt"
                      }
                    }
           

In [4]:
from local_llm_judge.search_backend import local_es

df, settings = local_es.search("brown sports coat", "m")
styled(df)

,name,brand_name,id,category,main_image,main_image_path,description,backend
0,Cotton-Corduroy Suit Jacket - Men,POLO RALPH LAUREN,541b437c-73b2-4908-b32e-825e4db59774,SUITS AND SEPARATES,,data/img/541b437c-73b2-4908-b32e-825e4db59774.png,"Polo Ralph Lauren’s suit jacket is inspired by a sport coat Mr. Lauren has had since 1971. Cut from fine-wale cotton-corduroy in a classic navy shade, it""s tailored with a button-fastening throat latch as part of the notch lapels and has practical front flap pockets. Wear yours with the matching trousers.",local_es
1,Furlong Wool Morning Coat - Men,FAVOURBROOK,f6adf44c-0227-4560-ace9-89163ddc7b13,SUITS AND SEPARATES,,data/img/f6adf44c-0227-4560-ace9-89163ddc7b13.png,"Favourbrook believes that ""wearing the correct attire for a special occasion needn""t mean losing one""s individual style and personality"". This traditional morning coat is tailored from wool and has lightly padded shoulders and peak lapels that broaden your frame. The hidden tail pocket is a traditional detail, originally used to hold gloves.",local_es
2,Check Tie in Cream,Burberry,6e37cd72-3422-44b0-af59-d51bd235a860,SUITS AND SEPARATES,,data/img/6e37cd72-3422-44b0-af59-d51bd235a860.png,"Burberry Check Tie in Cream 100% silk. Made in Italy. Dry clean only. Measures approx 2.75 in width. BURF-MA25. 8011693. About the designer: Founded in 1856, Burberry’s legacy as a pillar of British style grew from Thomas Burberry’s most iconic innovation: the gabardine trench coat. Upholding the pioneering spirit of his predecessor, current creative director Riccardo Tisci brings a fresh and irreverent approach to the house’s quintessentially British vision of luxury. Marrying technology, artistry, and tradition, the label touts a range of casual-wear and eveningwear that embodies a progressive femininity. Exquisite shirting, knitwear, and draped jackets exude refinement, while menswear-inspired polos, tailored trousers, and ankle boots achieve an easy polish.",local_es
3,self-tie wrap coat,SAINT LAURENT,af61c98f-f8db-44b9-948e-bf00513140e1,SUITS AND SEPARATES,,data/img/af61c98f-f8db-44b9-948e-bf00513140e1.png,black wrap design V-neck long sleeves turn-up cuffs self-tie fastening front pouch pocket,local_es
4,7cm Regular Tie in Blue,Burberry,f302a33e-e43a-4960-a631-f8bf5aeb81e7,SUITS AND SEPARATES,,data/img/f302a33e-e43a-4960-a631-f8bf5aeb81e7.png,"Burberry 7cm Regular Tie in Blue 100% silk. Made in England. Dry clean only. Measures approx 56 in length. BURF-MA112. 8103720. About the designer: Founded in 1856, Burberry’s legacy as a pillar of British style grew from Thomas Burberry’s most iconic innovation: the gabardine trench coat. Upholding the pioneering spirit of his predecessor, current creative director Riccardo Tisci brings a fresh and irreverent approach to the house’s quintessentially British vision of luxury. Marrying technology, artistry, and tradition, the label touts a range of casual-wear and eveningwear that embodies a progressive femininity. Exquisite shirting, knitwear, and draped jackets exude refinement, while menswear-inspired polos, tailored trousers, and ankle boots achieve an easy polish.",local_es
5,Exploded Check Tie in Brown,Burberry,34a3da3a-aa38-4dcf-8132-b6ddc68bd762,SUITS AND SEPARATES,,data/img/34a3da3a-aa38-4dcf-8132-b6ddc68bd762.png,"Burberry Exploded Check Tie in Brown 100% silk. Made in Italy. Dry clean only. BURF-MA71. 8013818. About the designer: Founded in 1856, Burberry’s legacy as a pillar of British style grew from Thomas Burberry’s most iconic innovation: the gabardine trench coat. Upholding the pioneering spirit of his predecessor, current creative director Riccardo Tisci brings a fresh and irreverent approach to the house’s quintessentially British vision of luxury. Marrying technology, artistry, and tradition, the label touts a range of casual-wear and eveningwear that embodies a progressive femininity. Exquisite shirting, knitwear, and draped jackets exude refinement, while menswear-inspired po

In [ ]:
local_es.last_rendered

In [15]:
from local_llm_judge.compare import compare_env
from local_llm_judge.log_stdout import enable

enable('local_llm_judge')

# Asking chat gpt to generate queries
chat_gpt_queries = [
    "Summer wedding guest dresses",
    "Men's slim-fit jeans with stretch fabric",
    "Designer handbags on sale",
    "Kids' rain boots",
    "Sustainable activewear brands",
    "Vintage leather jackets for women",
    "Plus-size evening gowns with sleeves",
    "Trendy sneakers for teenagers",
    "Men's formal wear rental options",
    "Boho-style maxi dresses with floral prints",
    "Athleisure wear for yoga and running",
    "Affordable cashmere sweaters for winter"
]

# Dougs queries
queries = ["adidas sambas", "red adidas sambas", "vegan leather jacket",
           "clutch purse", "blue clutch purse", 
           ("brown sports coat", "m")] + chat_gpt_queries

results = compare_env(queries, "cached_stag", "local_es")



2025-01-28 20:17:02,970 - local_llm_judge.compare - INFO - Loaded 1120 cached option pair evals
2025-01-28 20:17:02,970 - local_llm_judge.compare - INFO - Loaded 1120 cached option pair evals
2025-01-28 20:17:02,970 - local_llm_judge.compare - INFO - Comparing 18 queries
2025-01-28 20:17:02,970 - local_llm_judge.compare - INFO - Comparing 18 queries
2025-01-28 20:17:02,971 - local_llm_judge.compare - INFO - Processing Query: adidas sambas - Department: w
2025-01-28 20:17:02,971 - local_llm_judge.compare - INFO - Processing Query: adidas sambas - Department: w
2025-01-28 20:17:08,805 - local_llm_judge.search_backend - DEBUG - Search settings: {'search_string': 'sneakers', 'minimum_price': 0, 'maximum_price': 0, 'lexical_search': 'adidas sambas', 'image_search': 'sneakers', 'positive_filters': {'brand': ['adidas', 'ADIDAS', 'adidas'], 'color': [], 'department': ['w'], 'material': [], 'tag': [], 'category': ['shoes'], 'option_id': []}, 'negative_filters': {'brand': [], 'color': [], 'depar

error getting option details: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "option not found: no results were found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:35.222.239.43:443 {grpc_message:"option not found: no results were found", grpc_status:5, created_time:"2025-01-28T20:22:12.225614-05:00"}"
>


2025-01-28 20:22:12,226 - local_llm_judge.search_backend - ERROR - Error fetching option details for c011c27e-7ebb-469c-8205-aebfe78be77d: rpc framework returned error: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "option not found: no results were found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:35.222.239.43:443 {grpc_message:"option not found: no results were found", grpc_status:5, created_time:"2025-01-28T20:22:12.225614-05:00"}"
>
2025-01-28 20:22:12,226 - local_llm_judge.search_backend - ERROR - Error fetching option details for c011c27e-7ebb-469c-8205-aebfe78be77d: rpc framework returned error: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "option not found: no results were found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:35.222.239.43:443 {grpc_message:"option not found: no results were found", grpc_status:5, created_time:"2025-01-28T20:22:12.225614-05:

error getting option details: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "option not found: no results were found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:35.222.239.43:443 {grpc_message:"option not found: no results were found", grpc_status:5, created_time:"2025-01-28T20:22:12.300099-05:00"}"
>


2025-01-28 20:22:12,300 - local_llm_judge.search_backend - ERROR - Error fetching option details for f6bb44bf-4783-48f9-9f2d-33d5c98fcc18: rpc framework returned error: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "option not found: no results were found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:35.222.239.43:443 {grpc_message:"option not found: no results were found", grpc_status:5, created_time:"2025-01-28T20:22:12.300099-05:00"}"
>
2025-01-28 20:22:12,300 - local_llm_judge.search_backend - ERROR - Error fetching option details for f6bb44bf-4783-48f9-9f2d-33d5c98fcc18: rpc framework returned error: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "option not found: no results were found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:35.222.239.43:443 {grpc_message:"option not found: no results were found", grpc_status:5, created_time:"2025-01-28T20:22:12.300099-05:

In [19]:
import pandas as pd

def _url_to_img_tag(path):
    return f"""<img
    style="width: 100px; height: 100px; object-fit: contain;"
            src="{path}"/>"""


def styled(df: pd.DataFrame) -> pd.DataFrame:
    return df.style.format({'image_url_lhs': _url_to_img_tag,
                            'image_url_rhs': _url_to_img_tag})


styled(results[['query', 'pref_lhs', 'backend_lhs', 'name_lhs', 'brand_name_lhs', 'category_lhs', 'image_url_lhs',
                'name_rhs', 'brand_name_rhs', 'image_url_rhs', 'backend_rhs', 'pref_rhs']])

,query,pref_lhs,backend_lhs,name_lhs,brand_name_lhs,category_lhs,image_url_lhs,name_rhs,brand_name_rhs,image_url_rhs,backend_rhs,pref_rhs
0,adidas sambas,0.166667,stag,Samba 'Valentine's Day' sneakers,ADIDAS,SHOES,,Samba sneakers,ADIDAS,,local_es,0.833333
1,adidas sambas,0.401786,stag,"Samba Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Samba sneakers,ADIDAS,,local_es,0.598214
2,adidas sambas,0.401786,stag,"Samba Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Sambae '' sneakers,ADIDAS,,local_es,0.598214
3,adidas sambas,0.401786,stag,"Samba Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Sambae sneakers,ADIDAS,,local_es,0.598214
4,adidas sambas,0.401786,stag,"Samba Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Samba sneakers,ADIDAS,,local_es,0.598214
5,adidas sambas,0.494145,stag,Sambae Sneaker at Nordstrom,ADIDAS,SHOES,,Sambae Sneaker at Nordstrom,ADIDAS,,local_es,0.505855
6,adidas sambas,0.494145,stag,Samba XLG Sneaker at Nordstrom,ADIDAS,SHOES,,Sambae 'Core ' sneakers,ADIDAS,,local_es,0.505855
7,adidas sambas,0.401786,stag,"Gender Inclusive Samba OG Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Sambae Sneaker at Nordstrom,ADIDAS,,local_es,0.598214
8,adidas sambas,0.401786,stag,"Gender Inclusive Samba OG Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Sambae Sneaker at Nordstrom,ADIDAS,,local_es,0.598214
9,adidas sambas,0.401786,stag,"Gender Inclusive Samba OG Sneaker at Nordstrom, Women's",ADIDAS,SHOES,,Sambae '/Red' sneakers,ADIDAS,,local_es,0.598214


In [20]:
results['pref_lhs'].sum(), results['pref_rhs'].sum(), 

(117.9995542939141, 80.00044570608591)

In [ ]:
for _, row in results[results['query'] == 'vegan leather jacket'].iterrows():
    if row['pref_lhs'] > 0.4 and row['pref_lhs'] < 0.6:
        continue
    print("***")
    print(f"{row['name_lhs']} vs {row['name_rhs']}")
    print(f"{row['pref_lhs']} vs {row['pref_rhs']}")
    print(f"{row['brand_name_lhs']} vs {row['brand_name_rhs']}")
    print(f"{row['category_lhs']} vs {row['category_rhs']}")
    print()
    print("***")
    print(f"Desc LHS {row['desc_lhs']}")
    print(f"Desc rHS {row['desc_rhs']}")


    print('both_ways_desc', row['both_ways_desc'])
    # print(
    print('both_ways_category', row['both_ways_category'])
    print('both_ways_captions', row['both_ways_captions'])
    print('both_ways_brand', row['both_ways_brand'])
    print('both_ways_all_fields', row['both_ways_all_fields'])
    print()
    print()


In [ ]:
results.columns